<a href="https://colab.research.google.com/github/carlocamilloni/Structural-Bioinformatics/blob/main/Notebooks/t05_MD_enhanced_martini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Enhanced Sampling and Simplified Models in MD


In this lab experience you will perform two different simulations covering some more advanced Molecular Dynamics topic. Namely:


*   A metadynamics simulation of alanine dipeptide;
*   A self-assembly and equilibration simulation of a lipid bilayer using the Martini coarse-grain force-field.

The reference lectures are:

https://github.com/carlocamilloni/Structural-Bioinformatics/blob/main/Notes/02_StochasticMolecules.pdf
https://github.com/carlocamilloni/Structural-Bioinformatics/blob/main/Notes/03_MolecularDynamics.pdf
https://github.com/carlocamilloni/Structural-Bioinformatics/blob/main/Notes/04_EnhancedMD.pdf
https://github.com/carlocamilloni/Structural-Bioinformatics/blob/main/Notes/06_QM_MM_more.pdf

At odds with other Tasks, in this case instructions and reports are coupled. So GROUP 1 should focus on **Enhanced Sampling**, while GROUP 2 on **Martini simulations**.

# Enhanced Sampling

## Preliminary operations

To perform the tasks you first need to install some software:

The first step is to install the conda python package manager

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now we can install GROMACS (a molecular dynamics package) and PLUMED, that is a library to perform enhanced sampling techniques in combination with multiple MD packages, including gromacs (www.plumed.org). Be patient, the next step can take 5 minutes.

In [ ]:
!conda install --strict-channel-priority -c plumed/label/masterclass -c conda-forge gromacs

Now you are ready to start, so

1.   Make a copy of this notebook (File -> Save a copy in Drive)
2.   Mount your google drive
3.   Make a new folder for this task using the menus on the left side, for example in /drive/MyDrive/Structural_Bioinformatics make a folder named Task5
4.   In the Task5 folder make a folder named MTD




In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

In [ ]:
# set the path to your local task folder
mtd_path='/content/drive/MyDrive/Structural_Bioinformatics/Task5/MTD'
%cd {mtd_path}

In [ ]:
# test the gromacs executable
!gmx mdrun -h

In [ ]:
# make a copy of the GitHub structural bioinformatics repository
# this containts usefull input files for runnign the simulatoins
!if [ -d ../../sb/ ]; then (cd ../../sb/ && git pull); else git clone https://github.com/carlocamilloni/Structural-Bioinformatics.git ../../sb --depth 1; fi

##Metadynamics simulations on Alanine Dipetide in Vacuum




In a previous task you performed a conventional MD simulation of alanine dipeptide in vacuum. Now you can repeat the same simulation but using Metadynamics to enhance the sampling of conformational space with respect to some conformational parameters, in particular we will use the phi and psi angles.

###System preparation

First of all let's prepare the system as you already done before:

In [ ]:
%cd {mtd_path}
%cp ../../sb/Data/md/inputs/aladp.pdb .
%cp ../../sb/Data/md/mdps/0-em-steep.mdp em.mdp
%cp ../../sb/Data/md/mdps/run.mdp .

In [ ]:
# select force-field
!gmx pdb2gmx -f aladp.pdb -water none -ff amber03
# set the box
!gmx editconf -f conf.gro -bt cubic -d 1 -o boxed.gro
# energy minimisation
!gmx grompp -f em.mdp -c boxed.gro -p topol.top -o em.tpr
!gmx mdrun -deffnm em -v

In [ ]:
# From the energy minised file we also make a reference PDB structure that we can use later
!gmx editconf -f em.gro -o reference.pdb

In [ ]:
#now perform a preliminary 10 ns MD simulation, this will last less than 2 minutes
!gmx grompp -f run.mdp -c em.gro -p topol.top -o run.tpr
!gmx mdrun -deffnm run -v -ntomp 1

In [ ]:
#Now you should have multiple files, in particular run.gro and run.xtc that are
#the last frame and the trajectory from the MD simulation, respectively.
%ls

### Metadynamics

PLUMED allows both to analyze simulations trajectories by defining collective variables as well as then use CV based enhanced sampling techniques like Umbrella Sampling and Metadynamics to enhance the sampling of your MD. You can learn more on PLUMED syntax and features here: https://www.plumed.org/masterclass

Furthermore, you can find real-life simulations setups here: www.plumed-nest.org

As a first example let's analyse your MD simulation using plumed. To do so you need to write a plumed input file, that is a text file, including the definition of the quantity and the operation you want to perform.

You can now copy an example file and see its content:

In [ ]:
#copy the file in your mtd folder
%cp ../../sb/Data/md/metad/plumed-cv.dat .
#print the file on screen
%cat plumed-cv.dat

In [ ]:
#analyse your simulation using plumed executable and your input file
!plumed driver --plumed plumed-cv.dat --mf_xtc run.xtc

In [ ]:
# list files
%ls
# you should have a new COLVAR-CV.txt file

In [ ]:
#let's have a look at it:
#this print on screen only the first tops lines
!head COLVAR-CV.txt
# the file has some header lines reporting the content of the file, in this
# case is three columns reporting time, phi and psi actions that are defined
# in your plumed-cv.dat input file
# then it tells you that phi and psi are defined in some range of values
# then you have the actual data (angles are in radians here)

Now you can plot the data that should be comparable to what you observed in the former Task.

We plot both phi and psi as a function of the simulation time and we then we scatter plot phi,psi in a Ramachandran plot style

In [ ]:
import matplotlib.pyplot as plt
import csv

x=[]
y1=[]
y2=[]

with open('COLVAR-CV.txt', 'r') as file:
    for row in file:
        if row.split()[0] == '#!':
          continue
        x.append(float(row.split()[0]))
        y1.append(float(row.split()[1]))
        y2.append(float(row.split()[2]))

# Initialise the subplot function using number of rows and columns
figure, axis = plt.subplots(2,2)
figure.set_size_inches(18.5, 10.5)

axis[0,0].plot(x,y1,'.')
axis[0,0].set_ylabel("PHI")

axis[0,1].plot(x,y2,'.')
axis[0,1].set_ylabel("PSI")
axis[0,1].set_xlabel("Time (ps)")

axis[1,0].plot(y1,y2,'.')
axis[1,0].set_ylabel("PSI")
axis[1,0].set_xlabel("PHI")

plt.show()

Now we can run a Metadynamics simulation and see the difference! Let's first copy and have a look at the input file:

In [ ]:
%cp ../../sb/Data/md/metad/plumed-mtd.dat .
%cat plumed-mtd.dat

In the above input there is an action `metad` that takes in input the two torsion angles and set some parameters: `PACE` is how often update the histogram; it is the pace at which we learn from the simulation. `HEIGHT` and `SIGMA` are the parameters for the Gaussian that is used to build the histogram. `BIASFACTOR` is the dumping factor, we want our result to be the original one rescaled by a factor of 10, this means that if there is a barrier of 100 kJ/mol at the end it will be only 10 kJ/mol high. `GRID_MIN` and `GRID_MAX` set a grid over which accumulate the histogram.

We will start the simulation from the final configuration obtained from MD. The `-plumed plumed-mtd.dat` flag is telling gromacs to activate plumed by reading the plumed-mtd.dat input file.



In [ ]:
!gmx grompp -f run.mdp -c run.gro -p topol.top -o mtd.tpr

In [ ]:
# run it, expected to last around 5 minutes
!gmx mdrun -deffnm mtd -plumed plumed-mtd.dat -v -ntomp 1

In [ ]:
%ls

To visualise the simulation, you can copy the mtd.xtc and em.gro files on your workstation. Open the em.gro file with VMD and then load into the mtd.xtc file.

### Analysis

While biasing the simulations PLUMED has also performed the analysis on-the-fly of the dihedral angles and the bias, writing the output in the COLVAR-MTD.xtc file, so we can immediately plot it:

In [ ]:
#Preliminary look at the file:
!head COLVAR-MTD.txt

Here the fourth column reports the amount of Metadynamcs bias felt by the system at that time in that specific configuration

In [ ]:
import matplotlib.pyplot as plt
import csv

time=[]
phi=[]
psi=[]
bias=[]

with open('COLVAR-MTD.txt', 'r') as file:
    for row in file:
        if row.split()[0] == '#!':
          continue
        time.append(float(row.split()[0]))
        phi.append(float(row.split()[1]))
        psi.append(float(row.split()[2]))
        bias.append(float(row.split()[3]))

# Initialise the subplot function using number of rows and columns
figure, axis = plt.subplots(2,2)
figure.set_size_inches(18.5, 10.5)

axis[0,0].plot(time,phi,'.')
axis[0,0].set_ylabel("PHI")
axis[0,0].set_xlabel("Time (ps)")

axis[0,1].plot(time,psi,'.')
axis[0,1].set_ylabel("PSI")


axis[1,0].plot(phi,psi,'.')
axis[1,0].set_ylabel("PSI")
axis[1,0].set_xlabel("PHI")

axis[1,1].plot(time,bias,'.')
axis[1,1].set_xlabel("Time (ps)")
axis[1,1].set_ylabel("Bias (kj/mol)")

plt.show()

From the above plots it should be evident the speed up obtained it terms of sampling, but this is obtained adding additional energy to the system, the bias, as a consequence the resultign statistics is biased.

We can analyse the growth of the bias by looking at the evolution of the estimated free energy, this allows to recover the free energy as the opposite of the accumulated bias. For simplicity we look separately phi and psi

In [ ]:
%pwd
%cd ../MTD

In [ ]:
#this calculates the free energy as a function of phi,
#it will print multiple files fes-phi0.dat to fes-phi10.dat showing the evolution of the estimate
!plumed sum_hills --hills HILLS.txt --mintozero --outfile fes-phi --idw phi --kt 2.49 --stride 1000
#this calculates the free energy as a function of psi,
#it will print multiple files fes-psi0.dat to fes-psi10.dat showing the evolution of the estimate
!plumed sum_hills --hills HILLS.txt --mintozero --outfile fes-psi --idw psi --kt 2.49 --stride 1000
#this calculates the free energy as a function of phi and psi,
#it will print multiple a single fes-2D file showing the final estimate in 2D
!plumed sum_hills --hills HILLS.txt --mintozero --outfile fes-2D

In [ ]:
!head fes-phi10.dat

In [ ]:
import matplotlib.pyplot as plt
import csv

phi=[]
psi=[]
fes_phi=[]
fes_psi=[]

with open('fes-phi10.dat', 'r') as file:
    for row in file:
        if row.split()[0] == '#!':
          continue
        phi.append(float(row.split()[0]))
        fes_phi.append(float(row.split()[1]))

file.close()

with open('fes-psi10.dat', 'r') as file:
    for row in file:
        if row.split()[0] == '#!':
          continue
        psi.append(float(row.split()[0]))
        fes_psi.append(float(row.split()[1]))

file.close()

# Initialise the subplot function using number of rows and columns
figure, axis = plt.subplots(2,1)
figure.set_size_inches(18.5, 10.5)

axis[0].plot(phi,fes_phi)
axis[0].set_xlabel("PHI")
axis[0].set_ylabel("Free Energy (kj/mol)")

axis[1].plot(psi,fes_psi)
axis[1].set_xlabel("PSI")
axis[1].set_ylabel("Free Energy (kj/mol)")

plt.show()

#Martini Simulations

## Preliminary operations

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [1]:
#install the tool to generare Martini inputs from a structure (vermouth)
#install the tool to generate Martini solutions (insane)
!pip install mdtraj vermouth insane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 150.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 51.9 MB/s eta 0:00:00


In [1]:
!conda install --strict-channel-priority -c plumed/label/masterclass -c conda-forge gromacs

Channels:
 - plumed/label/masterclass
 - conda-forge
Platform: linux-64
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gromacs


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    certifi-2025.10.5          |     pyhd8ed1ab_0         156 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    fftw-3.3.10                |nompi_hf1063bd_110         2.0 MB  conda-forge
    gawk-5.3.1                 |       hcd3d067_0         1.1 MB  conda-forge
    gmp-6.3.0                  |       hac33072_2         449 KB  conda-forge
    gromacs-2019.6             |       h3fd9d12_0         7.8 MB  plumed/label/masterclass
    gsl-2.7                    |       he838d99_0         3.2 MB  conda-forge
    libasp

Now you are ready to start, so

1.   Make a copy of this notebook (File -> Save a copy in Drive)
2.   Mount your google drive
3.   Make a new folder for this task using the menus on the left side, for example in /drive/MyDrive/Structural_Bioinformatics make a folder named Task5
4.   In the Task5 folder make a folder named martini




In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [3]:
#make a new folder for this exercise, like
%cd /content/drive/MyDrive/Structural_Bioinformatics/Task5/
%mkdir martini
%cd martini
martini_path=%pwd

/content/drive/MyDrive/Structural_Bioinformatics/Task5
mkdir: cannot create directory ‘martini’: File exists
/content/drive/MyDrive/Structural_Bioinformatics/Task5/martini


In [4]:
# make a copy of the GitHub structural bioinformatics repository
# this containts usefull input files for runnign the simulatoins
!if [ -d ../../sb/ ]; then (cd ../../sb/ && git pull); else git clone https://github.com/carlocamilloni/Structural-Bioinformatics.git ../../sb --depth 1; fi

remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 50 (delta 34), reused 10 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (50/50), 1.34 MiB | 343.00 KiB/s, done.
From https://github.com/carlocamilloni/Structural-Bioinformatics
   263099c..4d3bae8  main       -> origin/main
Updating 263099c..4d3bae8
Fast-forward
 Data/martini/CK1d.pdb                        |   2570 +
 Data/martini/martini.itp                     | 356605 ++++++++++++++++++++++++
 Data/martini/protein_em.mdp                  |     20 +
 Data/martini/protein_eq.mdp                  |    105 +
 Data/martini/protein_md.mdp                  |    104 +
 Data/martini/water.itp                       |     19 +
 Notebooks/report_5_MD_enhanced_martini.ipynb |    119 +-
 Notebooks/t05_MD_enhanced_martini.ipynb      |    517 +-
 8 files changed, 359931 insertions(+), 128 deletions(-)
 create mode 100644 Data/martini/CK1d.pd

In [4]:
#copy all needed files
%cd {martini_path}
%cp ../../sb/Data/martini/* .

/content/drive/MyDrive/Structural_Bioinformatics/Task5/martini


## Lipids

We will begin with self-assembling a dipalmitoyl-phosphatidylcholine (DPPC) bilayer from a random configuration of lipids and water in the simulation box. The first step is to create a simulation box containing a random configuration of 128 DPPC lipids. This can be done by starting from a file containing a single DPPC molecule. A file with coordinates for a single DPPC molecule is available for you as DPPC-em.gro. The gromacs tool `insert-molecules` can take this single-molecule conformation and attempt to place it in a simulation box multiple times at a random position and random orientation, each time checking that there are no overlaps between the consecutively placed molecules.

In [ ]:
%ls

In [ ]:
!gmx insert-molecules -ci DPPC-em.gro -box 7.5 7.5 7.5 -nmol 128 -radius 0.21 -try 500 -o 128_noW.gro

The value of the flag `-radius` (default van der Waals radii) has been increased from its default atomistic length (0.105 nm) to a value reflecting the size of Martini CG beads. Here `noW` stands for no Water. Then we perform a short energy minimization of the system containing only the lipids; the only reason for doing this, is getting rid of high forces between beads that may have been placed too close to each other.

In [ ]:
!gmx grompp -f minimization.mdp -c 128_noW.gro -p dppc.top -o dppc-min-init.tpr
!gmx mdrun -deffnm dppc-min-init -v -c 128_minimized.gro

Then we use the gromacs tool `solvate`, add 6 CG waters per lipid (note that this corresponds to 24 all-atom waters per lipid).

In [ ]:
!gmx solvate -cp 128_minimized.gro -cs water.gro -o waterbox.gro -maxsol 768 -radius 0.21 -p dppc.top

Also here, the value of the flag `-radius` is used to reflect the size of Martini CG beads. The water beads are taken from the file water.gro provided for you. A new file, waterbox.gro is produced containing the 128 lipids and added water beads.

Now perform an energy minimization again:

In [ ]:
!gmx grompp -f minimization.mdp -c waterbox.gro -p dppc.top -o dppc-min-solvent.tpr
!gmx mdrun -deffnm dppc-min-solvent -v -c minimized.gro

Now, you are ready to run the self-assembly MD simulation. About 20 ns should suffice. The simulation can take around 10 minutes.

In [ ]:
!gmx grompp -f martini_md.mdp -c minimized.gro -p dppc.top -o dppc-md.tpr
!gmx mdrun -deffnm dppc-md -v -ntomp 2


In order to visualize the trajectory in VMD we need to perform some preprocessing:

In [ ]:
!echo "0" | gmx trjconv -s dppc-md.tpr -f dppc-md.xtc -pbc whole -dump 0 -o dppc-md-start.gro
!echo "0" | gmx trjconv -s dppc-md.tpr -f dppc-md.xtc -pbc whole -o dppc-md-viz.xtc

On your laptop you need the two output files (dppc-md-start.gro, dppc-md-viz.xtc) the topology file (dppc.top) and these additional files (cg_bonds.tcl, viz.vmd, martini_v2.1.itp, martini_v2.0_DPPC_01.itp)

IF YOU USE VMD ON A WINDOWS LAPTOP: before doing the next step you need to do to the following:

1.   Start VMD
2.   Menu Extension -> Tk Console
3.   In the console type `pwd` to check the folder in which VMD is running
4.   Using `cd ..` and `cd folder name` reach the folder in which you have all the lipids file
5.   Once you are there (check with `pwd`) you can proceed with the next steps



In VMD select the menu File -> open visualization and open the viz.vmd file. This should load the above files (if the file names are the same) with the correct visualization (the problem is that since Martini is a coarse-grained representation VMD does not understand automatically what the beads represent).

When the simulation has finished, check whether you got a bilayer.

If your bilayer was formed in a plane other than the xy-plane, rotate the system so that it will, so for example

In [ ]:
%cp dppc-md.gro dppc-md-xy.gro

#if you need to rotate around x: (remove the # from the one you need)
#!gmx editconf -f dppc-md.gro -rotate 90 0 0 -o dppc-md-xy.gro

#or if you need to rotate around y
#!gmx editconf -f dppc-md.gro -rotate 0 90 0 -o dppc-md-xy.gro

To get the (projected) area per lipid, you can simply divide the area of your simulation box by half the number of lipids in your system. The box-sizes can be obtained by running the gromacs tool `gmx energy` (ask for Box-X or Box-Y)

In [ ]:
#check the area per lipid
!echo "11\n\n" | gmx energy -f dppc-md.edr -o box-x-md.xvg

                      :-) GROMACS - gmx energy, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

AREA PER LIPID: __your answer__

To get the (projected) area per lipid, you can simply divide the area of your simulation box by half the number of lipids in your system. The box-sizes can be obtained by running the gromacs tool `gmx energy` (ask for Box-X or Box-Y) the box is cubic



The spontaneous assembly simulation was done using isotropic pressure coupling. The bilayer may have formed but is probably under tension because of the isotropic pressure coupling. Therefore, we first need to run a simulation in which the area of the bilayer can reach a proper equilibrium value. This requires that we use independent pressure coupling in the plane and perpendicular to the plane. Set up a simulation for another 20 ns at zero surface tension

In [ ]:
!gmx grompp -f martini_eq.mdp -c dppc-md-xy.gro -p dppc.top -o dppc-eq.tpr
!gmx mdrun -deffnm dppc-eq -v -ntomp 2

step 699900, remaining wall clock time:     0 s          
Writing final coordinates.

Back Off! I just backed up dppc-eq.gro to ./#dppc-eq.gro.1#
step 700000, remaining wall clock time:     0 s          
NOTE: 12 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:     1283.581      641.793      200.0
                 (ns/day)    (hour/ns)
Performance:     2827.083        0.008

GROMACS reminds you: "Like other defaulters, I like to lay half the blame on ill-fortune and adverse circumstances" (Mr. Rochester in Jane Eyre by Charlotte Bronte)



In [ ]:
#prepare files for visualisation (use viz-eq.vmd)
!echo "0" | gmx trjconv -s dppc-eq.tpr -f dppc-eq.xtc -pbc whole -dump 0 -o dppc-eq-start.gro
!echo "0" | gmx trjconv -s dppc-eq.tpr -f dppc-eq.xtc -pbc whole -o dppc-eq-viz.xtc

                     :-) GROMACS - gmx trjconv, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

In [ ]:
#check the new area per lipid
!echo "11\n\n" | gmx energy -f dppc-eq.edr -o box-x-eq.xvg

                      :-) GROMACS - gmx energy, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

## Proteins

Here you will follow the basic steps to setup a Martini simulation of a protein. In the Task you will observe the need when using Martini for proteins to introduce a structural bias in the form of an Elastic Network model.

In [5]:
#this will convert the structure in CK1d.pdb into the martini coarse grain rapresentation (CK1d_cg.pdb) and the martini force-field (CK1d_only.top)
#in this case the force field does not include the elastic network model
!martinize2 -f CK1d.pdb -x CK1d_cg.pdb -o CK1d_only.top -ff martini3001 -p backbone -dssp

    INFO - general - Read 1 molecules from PDB file CK1d.pdb
    INFO - step - Guessing the bonds.
    INFO - general - 1 molecules after guessing bonds
    INFO - step - Repairing the graph.
    INFO - general - Applying modification N-ter to residue A-LEU3
    INFO - general - Applying modification + to residue A-LEU3
    INFO - general - Applying modification C-ter to residue A-ARG299
    INFO - general - Applying modification + to residue A-ARG299
    INFO - step - Dealing with modifications.
    INFO - general - Identified the modifications ['N-ter'] on residues ['LEU3', 'LEU3', 'LEU3', 'LEU3']
    INFO - general - Identified the modifications ['C-ter'] on residues ['ARG299', 'ARG299', 'ARG299']
    INFO - step - Read input.
    INFO - step - Creating the graph at the target resolution.
    INFO - general - Applying modification mapping ('C-ter',)
    INFO - general - Applying modification mapping ('N-ter',)
    INFO - step - Averaging the coordinates.
    INFO - step - Applying t

In [6]:
#to solvate the system:
!gmx editconf -f CK1d_cg.pdb -o CK1d_cg.gro -box 10 10 10 -c
!gmx solvate -cp CK1d_cg.gro -cs water.gro -o CK1d_cg_water.gro -maxsol 8400 -radius 0.21 -p CK1d_only.top
#we finally need to edit the topology file to include the water parameters:
#with this bash command we insert #include "water.itp" after the line #include "martini.itp"
!{ head -n 2 CK1d_only.top; echo '#include "water.itp"'; tail -n +3 CK1d_only.top; } > tmp && mv tmp CK1d_only.top

                     :-) GROMACS - gmx editconf, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, B

In [7]:
#run an energy minimization
!gmx grompp -p CK1d_only.top -c CK1d_cg_water.gro -f protein_em.mdp -o CK1d_em.tpr -r CK1d_cg_water.gro
!gmx mdrun -deffnm CK1d_em -v

Streaming output truncated to the last 5000 lines.
Step= 3702, Dmax= 3.4e-03 nm, Epot= -2.17525e+05 Fmax= 2.03864e+02, atom= 658
Step= 3703, Dmax= 4.0e-03 nm, Epot= -2.17527e+05 Fmax= 1.98154e+02, atom= 658
Step= 3704, Dmax= 4.8e-03 nm, Epot= -2.17528e+05 Fmax= 2.87790e+02, atom= 658
Step= 3705, Dmax= 5.8e-03 nm, Epot= -2.17530e+05 Fmax= 2.90094e+02, atom= 658
Step= 3706, Dmax= 7.0e-03 nm, Epot= -2.17530e+05 Fmax= 4.11943e+02, atom= 658
Step= 3707, Dmax= 8.4e-03 nm, Epot= -2.17531e+05 Fmax= 4.18556e+02, atom= 658
Step= 3709, Dmax= 5.0e-03 nm, Epot= -2.17536e+05 Fmax= 8.02622e+01, atom= 658
Step= 3711, Dmax= 3.0e-03 nm, Epot= -2.17539e+05 Fmax= 2.24053e+02, atom= 658
Step= 3712, Dmax= 3.6e-03 nm, Epot= -2.17541e+05 Fmax= 1.39080e+02, atom= 658
Step= 3713, Dmax= 4.3e-03 nm, Epot= -2.17542e+05 Fmax= 2.91828e+02, atom= 658
Step= 3714, Dmax= 5.2e-03 nm, Epot= -2.17544e+05 Fmax= 2.29696e+02, atom= 658
Step= 3715, Dmax= 6.2e-03 nm, Epot= -2.17544e+05 Fmax= 3.90198e+02, atom= 658
Step= 3716, D

In [11]:
#run a short equilibration MD simulation
!gmx grompp -p CK1d_only.top -c CK1d_em.gro -f protein_eq.mdp -o CK1d_eq.tpr -r CK1d_cg_water.gro -maxwarn 1
!gmx mdrun -deffnm CK1d_eq -v

                      :-) GROMACS - gmx grompp, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

In [12]:
#run a short production MD simulation
!gmx grompp -p CK1d_only.top -c CK1d_eq.gro -f protein_md.mdp -o CK1d_md.tpr -maxwarn 1
!gmx mdrun -deffnm CK1d_md -v

                      :-) GROMACS - gmx grompp, 2019.6 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Be

### With the Elastic Network Model

In [ ]:
#to generate instead a topology including the elastic network term you can use the -elastic additional flag:
!martinize2 -f CK1d.pdb -x CK1d_cg.pdb -o CK1d_enm_only.top -ff martini3001 -p backbone -dssp -elastic -el 0 -eu 0.85

and then repeat all steps, possibly changing the names of the output files to avoid overwriting the files generated before.